In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
from tabulate import tabulate

In [24]:
import pandas as pd
from tabulate import tabulate

# Read the first CSV file and drop specified columns
df = pd.read_csv("Excels\\Excel-1 - District-Level_GWQ_AllYears.csv")
drop_columns = 'arsenic,carbonate,chloride,fluorine,iron,hydrogencarbonate,potassium,magnesium,nitrate,sodium,percentageofsodium,phosphateion,residualsodiumcarbonate,sodiumabsorptionratio,sulfate,silicondioxide,alkalinitytotal,potentialofhydrogen'.split(',')
df = df.drop(columns=drop_columns)
df['SDP'] = None
# Display the modified DataFrame
table = tabulate(df, headers='keys', tablefmt='pretty')

df2 = pd.read_csv("Excels\\standardised1999-2022.csv")
df2['exact_year'] = df2['YEAR'].str.split('-').str[0]
df2.insert(1, 'exact_year', df2.pop('exact_year'))
df2 = df2.rename(columns={'JAMMU & KASHMIR': 'JAMMU AND KASHMIR'})
df2 = df2.rename(columns={'ORISSA': 'ODISHA'})
df2 = df2.rename(columns={'TAMIL NADU': 'TAMILNADU'})
df2 = df2.rename(columns={'ANDAMAN & NICOBAR ISLANDS': 'ANDAMAN AND NICOBAR ISLANDS'})
df2 = df2.rename(columns={'PUDUCHERRY': 'PONDICHERRY'})


table = tabulate(df2, headers='keys',     tablefmt='pretty')
exact_years = df2['SIKKIM']


value = df2.loc[(df2['exact_year'] == '2000') & (df2['ANDHRA PRADESH'].notnull()), 'ANDHRA PRADESH'].values
    # return value[0] if len(value) > 0 else None


# print(table)
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    year = row['year']
    state = row['state'].upper()
    if (state=='TAMIL NADU'): state='TAMILNADU'
    if ( state=='THE DADRA AND NAGAR HAVELI AND DAMAN AND DIU'): continue

    # if (df2[state].notnull==False): continue
    value =( df2.loc[(df2['exact_year'] == str(year)) & (df2[state].notnull()), state].values)
    
    if (len(value)==0): continue
    df.at[index, 'SDP'] = float(value[0])
    
    # Do something with year and state (for example, print them)
    


In [25]:
table = tabulate(df2, headers='keys',     tablefmt='pretty')
print(table)

+----+---------+------------+----------------+-------------------+-------------+-------------+-------------+-------------+-------------+------------------+-------------------+-------------+-------------+-------------+----------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+---------------+-------------+-------------+-----------------------------+-------------+-------------+-------------+-------------+
|    |  YEAR   | exact_year | ANDHRA PRADESH | ARUNACHAL PRADESH |    ASSAM    |    BIHAR    |     GOA     |   GUJARAT   |   HARYANA   | HIMACHAL PRADESH | JAMMU AND KASHMIR |  JHARKHAND  |  KARNATAKA  |   KERALA    | MADHYA PRADESH | CHHATTISGARH | MAHARASHTRA |   MANIPUR   |  MEGHALAYA  |   MIZORAM   |  NAGALAND   |   ODISHA    |   PUNJAB    |  RAJASTHAN  |   SIKKIM    |  TAMILNADU  |   TRIPURA   | UTTAR PRADESH | UTTARAKHAND | WEST BENGAL | ANDAMAN AND NICOBA

In [26]:
table = tabulate(df, headers='keys', tablefmt='pretty')
print("Modified DataFrame from Excel-1:")
print(table)

Modified DataFrame from Excel-1:
+-------+---------+----------------------------------------------+-----------------------------+------+---------------------------------+-----------+------------------------+---------------+----------------------+-------------+
|       | country |                    state                     |          district           | year |              dyid               |  calcium  | electricalconductivity | hardnesstotal | totaldissolvedsolids |     SDP     |
+-------+---------+----------------------------------------------+-----------------------------+------+---------------------------------+-----------+------------------------+---------------+----------------------+-------------+
|   0   |  India  |                 West Bengal                  |     24 Paraganas North      | 2000 |     24 Paraganas North2000      | 50.588234 |       848.47058        |   261.76471   |         nan          |  129691.0   |
|   1   |  India  |                 West Bengal        